In [1]:
!pip install nltk
!pip install pandas
!pip install numpy
import nltk
nltk.download('stopwords')
nltk.download('punkt')
!pip install ipython-autotime

%load_ext autotime

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer

import nltk
import os
import string
import numpy as np
import copy
import pandas as pd
import pickle

time: 197 ms


# Data Extraction

In [3]:
title = "20_newsgroups"

time: 368 µs


In [4]:
paths = []
for (dirpath, dirnames, filenames) in os.walk(str(os.getcwd())+'/'+title+'/'):
    for i in filenames:
        paths.append(str(dirpath)+str("/")+i)

time: 52.5 ms


In [5]:
paths[0]

'/root/my_project_dir/20_newsgroups/talk.politics.guns/54541'

time: 9.94 ms


In [6]:
# paths.remove(paths[0]) #my computer is generating a dummy file, removing that!

time: 4.4 ms


In [7]:
len(paths)

19997

time: 2.36 ms


# Preprocessing

In [8]:
def remove_header(data):
    try:
        ind = data.index('\n\n')
        data = data[ind:]
    except:
        print("No Header")
    return data

time: 1.29 ms


In [9]:
def convert_lower_case(data):
    return np.char.lower(data)

time: 594 µs


In [10]:
def remove_stop_words(data):
    stop_words = stopwords.words('english')
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if w not in stop_words:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 1.31 ms


In [11]:
def remove_punctuation(data):
    symbols = "!\"#$%&()*+-./:;<=>?@[\]^_`{|}~\n"
    for i in range(len(symbols)):
        data = np.char.replace(data, symbols[i], ' ')
        data = np.char.replace(data, "  ", " ")
    data = np.char.replace(data, ',', '')
    return data

time: 1.51 ms


In [12]:
def remove_apostrophe(data):
    return np.char.replace(data, "'", "")

time: 947 µs


In [13]:
def remove_single_characters(data):
    words = word_tokenize(str(data))
    new_text = ""
    for w in words:
        if len(w) > 1:
            new_text = new_text + " " + w
    return np.char.strip(new_text)

time: 1.42 ms


In [14]:
def convert_numbers(data):
    data = np.char.replace(data, "0", " zero ")
    data = np.char.replace(data, "1", " one ")
    data = np.char.replace(data, "2", " two ")
    data = np.char.replace(data, "3", " three ")
    data = np.char.replace(data, "4", " four ")
    data = np.char.replace(data, "5", " five ")
    data = np.char.replace(data, "6", " six ")
    data = np.char.replace(data, "7", " seven ")
    data = np.char.replace(data, "8", " eight ")
    data = np.char.replace(data, "9", " nine ")
    return data

time: 2.63 ms


In [15]:
def stemming(data):
    stemmer= PorterStemmer()
    
    tokens = word_tokenize(str(data))
    new_text = ""
    for w in tokens:
        new_text = new_text + " " + stemmer.stem(w)
    return np.char.strip(new_text)

time: 7.68 ms


In [16]:
def preprocess(data, query):
    if not query:
        data = remove_header(data)        
    data = convert_lower_case(data)
    data = convert_numbers(data)
    data = remove_punctuation(data) #remove comma seperately
    data = remove_stop_words(data)
    data = remove_apostrophe(data)
    data = remove_single_characters(data)
    data = stemming(data)
    return data

time: 1.32 ms


# Generating Postings for Unigram inverted index

In [17]:
doc = 0
postings = pd.DataFrame()

for path in paths:
    file = open(path, 'r', encoding='cp1250')
    text = file.read().strip()
    file.close()
    preprocessed_text = preprocess(text, False)
    
    if doc%100 == 0:
        print(doc)

    tokens = word_tokenize(str(preprocessed_text))
    for token in tokens:
        if token in postings:
            p = postings[token][0]
            p.add(doc)
            postings[token][0] = p
        else:
            postings.insert(value=[{doc}], loc=0, column=token)
    doc += 1

0
100
200
300
400
500
600
700
800
900
1000
1100
1200
No Header
1300
1400
No Header
1500
1600
1700
1800
1900
No Header
2000
2100
2200
2300
2400
2500
2600
No Header
2700
2800
2900
3000
3100
3200
3300
No Header
3400
3500
3600
3700
3800
No Header
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
No Header
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
No Header
6400
6500
6600
6700
6800
6900
7000
No Header
7100
No Header
7200
7300
No Header
7400
7500
7600
7700
No Header
7800
7900
8000
8100
8200
8300
8400
No Header
8500
8600
8700
No Header
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
No Header
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
No Header
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
No Header
13300
13400
13500
13600
13700
13800
13900
No Header
14000
14100
No Header
No Header
No Header
No Header
14200
14300
14400
No Header
14500
14600


In [18]:
postings

,midweek,grafx,tasco,stoneham,montval,homewrit,jourdain,turley,dahmk,foosum,...,forev,wait,demand,nay,will,mani,fact,comment,notic,anyon
0,{19996},{19994},{19990},{19989},{19989},{19985},{19984},{19984},{19984},{19984},...,"{0, 6150, 16904, 14, 3598, 10766, 10769, 536, ...","{0, 8194, 12291, 8200, 8203, 10252, 14, 14351,...","{0, 5121, 15361, 3, 11268, 15873, 5639, 15370,...","{0, 15241, 14, 16787, 15639, 4637, 3761, 1075,...","{0, 8194, 14341, 2054, 11, 4109, 14, 15, 12304...","{0, 8192, 4, 5, 6, 8196, 14, 17, 25, 8218, 27,...","{0, 1, 16391, 8, 14, 15, 17, 8209, 16404, 22, ...","{0, 8192, 14336, 10243, 10245, 12294, 2055, 16...","{0, 12291, 10246, 10, 4107, 2061, 14, 18445, 1...","{0, 16385, 4, 8, 16393, 16394, 11, 8203, 16397..."


time: 35.8 ms


In [19]:
postings.to_pickle(title + "_unigram_postings")

time: 179 ms


In [20]:
postings = pd.read_pickle(title + "_unigram_postings")

time: 506 ms


# (complex)Search Query for Unigram Index

In [21]:
def get_word_postings(word):
    preprocessed_word = str(preprocess(word, True))
    print(preprocessed_word)
    print("Frequency:",len(postings[preprocessed_word][0]))
    print("Postings List:",postings[preprocessed_word][0])

time: 827 µs


In [22]:
def get_not(word):
    a = postings[word][0]
    b = set(range(len(paths)))
    return b.difference(a)

time: 971 µs


In [23]:
def generate_command_tokens(query):
    query = query.lower()
    tokens = word_tokenize(query)

    commands = []
    query_words = []

    for t in tokens:
        if t not in ['and', 'or', 'not']:
            processed_word = preprocess([t], True)
            print(str(processed_word))
            query_words.append(str(processed_word))
        else:
            commands.append(t)
    
    return commands, query_words

time: 1.21 ms


In [24]:
def gen_not_tuple(query_words, commands):
    tup = []
    while 'not' in commands:
        i = commands.index('not')
        word = query_words[i]
        word_postings = get_not(word)
        tup.append(word_postings)
        commands.pop(i)
        query_words[i] = i
        print("\nAfter Not Processing:",commands, query_words)
    return tup

time: 1.26 ms


In [25]:
def binary_operations(query_words, commands, tup):
    a = postings[query_words[0]][0]
    query_words.pop(0)

    for i in range(len(commands)):
        if type(query_words[i])==int:
            b = tup.pop(0)
        else:
            b = postings[query_words[i]][0]

        if commands[i] == 'and':
            a = a.intersection(b)
        elif commands[i] == 'or':
            a = a.union(b)
        else:
            print("Invalid Command")
            
    return a

time: 1.46 ms


In [26]:
def execute_query(query):

    commands, query_words = generate_command_tokens(query)
    tup = gen_not_tuple(query_words, commands)
    
    print("\nCommands:",commands)
    print("\nQuery Words:",query_words)
    print("\nTup:",len(tup))
    
    final_set = binary_operations(query_words, commands, tup)
    
    print("\nFinal Set:",final_set)
    return final_set

time: 1.12 ms


In [27]:
def print_file(file):
    out_file = open(paths[file], 'r', encoding='cp1250')
    out_text = out_file.read()
    print(out_text)

time: 1.09 ms


In [28]:
query = "place and inhumane and not authenticity and not hell"

time: 1.05 ms


In [29]:
lists = execute_query(query)

place
inhuman
authent
hell

After Not Processing: ['and', 'and', 'and', 'not'] ['place', 'inhuman', 2, 'hell']

After Not Processing: ['and', 'and', 'and'] ['place', 'inhuman', 2, 3]

Commands: ['and', 'and', 'and']

Query Words: ['place', 'inhuman', 2, 3]

Tup: 2

Final Set: {5349, 5131, 845, 5550, 5331, 5881, 5463, 5624, 5817, 5438}
time: 30.2 ms


In [30]:
get_word_postings('welcome')

welcom
Frequency: 314
Postings List: {9216, 17925, 1032, 15371, 17421, 8718, 7695, 17422, 5649, 16917, 4119, 4122, 14876, 10781, 18974, 5156, 11815, 9768, 18473, 7722, 9259, 4652, 15406, 3631, 17456, 561, 3633, 6195, 6708, 7217, 7731, 572, 9789, 14908, 13887, 14912, 19521, 6211, 5700, 12867, 13385, 11338, 15949, 2130, 4179, 596, 17490, 11862, 14938, 92, 10846, 2143, 17504, 609, 10851, 103, 11879, 2153, 17001, 18027, 624, 1137, 7797, 17525, 13432, 17530, 4730, 17532, 17021, 1661, 13438, 4737, 6273, 13953, 14979, 15491, 9350, 11260, 2184, 3208, 5770, 17547, 3720, 11918, 16527, 9362, 13975, 16537, 9882, 667, 11932, 16541, 17565, 13983, 19099, 11427, 8868, 13477, 9382, 5288, 3755, 12459, 17073, 18609, 5815, 7351, 18617, 18619, 10941, 13501, 19646, 16576, 5316, 6340, 16581, 11978, 16074, 14032, 11985, 14034, 11475, 10452, 15569, 13526, 15062, 729, 10459, 3804, 11484, 15070, 16607, 8928, 19169, 17123, 8933, 13541, 10471, 17126, 13545, 19176, 8427, 11500, 13035, 10992, 14064, 15605, 19189, 33

In [31]:
print_file(84)

Newsgroups: talk.politics.guns
Path: cantaloupe.srv.cs.cmu.edu!magnesium.club.cc.cmu.edu!news.sei.cmu.edu!cis.ohio-state.edu!zaphod.mps.ohio-state.edu!darwin.sura.net!paladin.american.edu!europa.eng.gtefsd.com!emory!rsiatl!karl
From: karl@dixie.com (Karl Klingman)
Subject: Re: The Truth about Waco 
Message-ID: <_+tvn0k@dixie.com>
Date: Mon, 26 Apr 93 19:40:52 GMT
Organization: Dixie Communications Public Access.  The Mouth of the South.
Distribution: usa
References: <25337@alice.att.com> <1ppv94-@dixie.com> <C6054E.BJq@chinet.chi.il.us>
Lines: 74

dhartung@chinet.chi.il.us (Dan Hartung) writes:

>jgd@dixie.com (John De Armond) writes:

>>*	The tanks were collapsing interior walls and ceilings putting people
>>	at great risk.

>Dear, dear. They could have COME OUT.

Then by your logic, the Jews in Europe in the 1930's were the cause for
the Holocaust.  Hitler told them to leave and because they didn't they
brought the whole thing on themselves.  Because as you say, they could
have COME 